In [ ]:
import cv2
import numpy as np
from lib.ev3 import EV3

import tensorflow.keras
from tensorflow.keras.models import Model, load_model

def main():
    capture = cv2.VideoCapture(0)
    if capture.isOpened() is False:
        raise('IO Error')

    """ev3 setting"""
    # ev3 port
    lmotor_port = EV3.PORT_B
    rmotor_port = EV3.PORT_C
    touch_port = EV3.PORT_2

    # define ev3 motor
    ev3 = EV3()
    ev3.motor_config(lmotor_port, EV3.LARGE_MOTOR)
    ev3.motor_config(rmotor_port, EV3.LARGE_MOTOR)
    ev3.sensor_config(touch_port, EV3.TOUCH_SENSOR)

    # initialize value
    throttle = 0
    steer = 0

    image_width = 160
    image_height = 120



    # load model path
    model = load_model('./14_model.h5', compile = True )


    while True:
        # getImage
        ##reshaped_imgに160×120の画像データを格納
        ret, frame = capture.read()
        reshaped_img = cv2.resize(frame, (image_width, image_height))

       # End condition
        if ev3.touch_sensor_is_pressed(touch_port):
            ev3.motor_steer(lmotor_port, rmotor_port, 0, 0)
            break

        #Inputの形に適応させて、正規化
        reshaped_img = reshaped_img[np.newaxis,:,:]/255.0
        ##画像データの行列配列の一列目に新しい行を追加している。行ベクトルを作成
        predict = model.predict_on_batch(reshaped_img)

        steer = predict[0][0].squeeze()
        throttle = predict[1][0].squeeze()

        if steer >=50 :
            steer = 50
        elif steer <= -50:
            steer = -50



        print('steer={}, throttle={}'.format(steer, throttle))

        # controle ev3
        ev3.motor_steer(lmotor_port, rmotor_port, throttle, steer)

main()


VIDEOIO ERROR: V4L: index 1 is not correct!


TypeError: exceptions must derive from BaseException